# CoF<sub>2</sub>
In this example we show how to calculate the field at the known muon site in the Néel antiferromagnetic insulator CoF<sub>2</sub>

In [ ]:
%pip install matplotlib
import numpy as np
from muesr.engines.clfc import locfield # Does the sum and returns the results
from muesr.core import Sample           # The object that contains the information
from muesr.engines.clfc import find_largest_sphere # A sphere centered at the muon is the correct summation domain
from muesr.i_o import load_cif          # To load crystal structure information from a cif file
from muesr.utilities import mago_add, show_structure # To define the magnetic structure and show it
import matplotlib.pyplot as P           
np.set_printoptions(suppress=True,precision=4)       # to set displayed decimals in results

You can find all relevant MuSR information of this compound 
in <a href="https://doi.org/10.1103/PhysRevB.30.186">Phys. Rev. B 30 186</a></center>.

Now define a sample object (call it `cof` for short) and load the lattice structure from a CIF file 
(present in the muesr distribution). Finally add the known muons site in fractional cell coordinates 
(it sits in the middle of the $a$ axis).

In [ ]:
cof = Sample()
load_cif(cof,"./CoF2.cif")
cof.add_muon([0.5,0.0,0.0]) # Checked experimentally by single crystal studies in external field

## Propagation Vector Formalism

Now define a new magnetic structure (you can have more than one). To do so we use the propagation
vector formalism. This is the basis a complete and robust description and categorization of magnetic orders, but we will not touch this aspect in this tutorial.

We want to describe magnetic orders on the basis of crystallographic unit cells, although their repetition units are not necessarily the same. The key idea is that one can define just the spins in a reference crystallographic unit cell (with index 0) and use the k-vector to describe how the spins are directed in all the other cells.

$$
\mathbf{m}_{j}=\sum_{\left\{ \mathbf{k}\right\} }\mathbf{S}_{j} ^{\mathbf{k}} e^{-i\mathbf{k\cdot}\mathbf{T}} =\sum _\mathbf{k} \mathbf{S}_{j} ^{\mathbf{k}} [\cos(-\mathbf{k} \cdot \mathbf{T})+i \sin(-\mathbf{k} \cdot \mathbf{T}) ]
$$

In the equation above

* $\mathbf{m}_j$ is the magnetic moment of atomic site $j$ in the unit cell identified by the lattice translation vector $\mathbf{R}$.
* $\mathbf{S}_j$ is the Fourier (or basis) component on atom $j$ defined in the unit cell. These are complex vectors.
* $\mathbf{k}$ is the propagation vector

In some cases we need more than a propagation vector (and the associated Fourier components). The most general description of magnetic orders is provided by the sum over $\{\mathbf{k}\}$ vectors shown above.

The moment $\mathbf{m}_j$ must be real. The simplest case is 
that of $k =(0,0,0)$ that we consider today. From this we get

$$
\mathbf{m}_j = \mathbf{S}_j e^{-i \mathbf{k} \cdot \mathbf{R}} = \mathbf{S}_j e^{-i 0 \cdot \mathbf{R}} = \mathbf{S}_j
$$

Therefore $\mathbf{S}_j$ must be real and can be directly identified with the moment of the magnetic atoms in the unit cell.

More information on the description of magnetic orders can be found in [Magnetic structures and their determination using group theory, A. Wills (2001)](http://jp4.journaldephysique.org/index.php?option=com_article&access=doi&doi=10.1051/jp4:2001906&Itemid=129) (DOI: [10.1051/jp4:2001906](http://dx.doi.org/10.1051/jp4:2001906)).

## Apply this to CoF2

Here is the magnetic structure of CoF<sub>2</sub> (from [PRB 30 186](https://doi.org/10.1103/PhysRevB.30.186))

![title](CoF2.png)

The antiferromagnetic order of CoF2 can be entirely defined within the unit-cell so we define its propagation vector <i><b>k</b></i> as (0,0,0).

The next step will be to input one complex Fourier component per atom, by the command `cof.mm.fc` 
You must know the order in which the atoms are presented. You can use `mago_add` for a simpler and interactive description of the magnetic order.


In [ ]:
mago_add(cof)

# Alternatively, you can write everything into code.

# magnetic moment of 2.6 muB from https:doi.//org/10.1103/PhysRevB.87.121108  https://doi.org/10.1103/PhysRevB.69.014417
cof.new_mm()     
cof.mm.k=np.array([0.0,0.0,0.0])
# according to CoF2.cif (setting with a,b equal, c shorter, type cif to check)
# H-M P4_2/mnm group 136, six atoms in the cell, in this order
# Co at 0.00000 0.00000 0.00000 (2b site)  
# the symmetry replica is generated at 0.5000 0.5000 0.5000
# http://www.cryst.ehu.es/cgi-bin/cryst/programs/nph-normsets?&norgens=&gnum=136 
# F at 0.30600 0.30600 0.00000  (4f site)
# the symmetry replicas are generated at 1--x 1-x 0, 0.5+x 0.5-x, 0.5, 0.5-x,0.5+x, 0.5
cof.mm.fc= np.array([[0.0+0.j, 0.0+0.j, 2.6+0.j],[0.0+0.j, 0.0+0.j, -2.6+0.j], # the two Co with opposite m
                    [0.0+0.j, 0.0+0.j, 0.0+0.j],[0.0+0.j, 0.0+0.j, 0.0+0.j],   # F
                    [0.0+0.j, 0.0+0.j, 0.0+0.j],[0.0+0.j, 0.0+0.j, 0.0+0.j]])  # F

**(NB: this only works when working with a local installation)**

Let us see if we did that right: invoke VESTA 
(you must have it installed and its location known to muesr, see Installation). And remember to kill VESTA to proceed. 

In [ ]:
show_structure(cof,visualizationTool='V') # show_structure(cof,supercell=[1,1,2],visualizationTool='V')

CoF<sub>2</sub> does not have a contact hyperfine term, by symmetry, nor a macroscopic magnetization to produce a demagnetizing field, 
being an antiferromagnet. We can just proceed to calculate the dipolar sums. 
Let us do that over a pretty large spherical summation domain.

In [ ]:
n=100
radius=find_largest_sphere(cof,[n,n,n]) 
r=locfield(cof, 's', [n, n, n] ,radius)
print('Compare Bdip = {:.4f} T with the experimental value of Bexp = 0.265 T'.format(np.linalg.norm(r[0].D,axis=0)))

The small 1.4% difference may be due to the muon pushing slightly the nearest neighbor Co ions. 
Now let us check the convergence and plot it

In [ ]:
npoints = 11
n = np.logspace(0.53,2,npoints,dtype=int)
k = -1
B_dip = np.zeros(npoints)
R = np.zeros(npoints)
for m in n:
    k += 1
    radius=find_largest_sphere(cof,[m,m,m])
    r=locfield(cof, 's', [m, m, m] ,radius) #
    R[k] = radius
    B_dip[k] = np.linalg.norm(r[0].D,axis=0)

fig,ax = P.subplots()

ax.plot(R,B_dip,'bo',label='sum')
ax.plot(R,R-R+0.265,'b--',label='exp')
ax1 = ax.twinx()
ax.set_xlabel('R')
ax.set_ylabel(r'$B_d$  [T]')
ax1.plot(R,n,'rd')
ax1.set_ylabel('m (sites per cube edge)')
ax.legend(loc=9)
P.show()

Compare with 

![title](examples_CoF2_CoF2_11_0.png)